# Laboratorio 3

En este laboratorio nuevamente utilizaremos los datos de COVID-19 disponibilizados por el Ministerio de Ciencias, Tecnología, Conocimiento e Innovación de Chile ([info](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto1)).

In [1]:
import pandas as pd

covid = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19_std.csv")
covid.head()

Region  Codigo region                          Comuna  \
0  Arica y Parinacota             15                           Arica   
1  Arica y Parinacota             15                       Camarones   
2  Arica y Parinacota             15                   General Lagos   
3  Arica y Parinacota             15                           Putre   
4  Arica y Parinacota             15  Desconocido Arica y Parinacota   

   Codigo comuna  Poblacion       Fecha  Casos confirmados  
0        15101.0   247552.0  2020-03-30                6.0  
1        15102.0     1233.0  2020-03-30                0.0  
2        15202.0      810.0  2020-03-30                0.0  
3        15201.0     2515.0  2020-03-30                0.0  
4            NaN        NaN  2020-03-30                NaN

## Ejercicio 1

(1 pto)

¿Cuál es la región que tiene menos registros (filas)?

In [2]:
covid["Region"].value_counts()

Metropolitana         2915
Valparaíso            2145
Biobío                1870
O’Higgins             1870
Araucanía             1815
Los Lagos             1705
Maule                 1705
Ñuble                 1210
Coquimbo               880
Los Ríos               715
Magallanes             660
Aysén                  605
Antofagasta            550
Atacama                550
Tarapacá               440
Arica y Parinacota     275
Name: Region, dtype: int64

__Respuesta:__ La región de Arica y Parinacota

## Ejercicio 2

(2 puntos)

1. Define la serie `casos` tal que:
    * Solo sean los registros de la región Valparaíso.
    * Los _index_ deben ser la _Fecha_ pero en formato `datetime`. Hint: `pd.to_datetime()`
    * Los _values_ deben ser los _Casos confirmados_.
2. Define la serie `casos_not_null` filtrando los calores `NaN` de la serie `casos`.
3. Obtén el mínimo, máximo y suma y promedio de la serie `casos_not_null`.

In [3]:
covid_valpo = covid.query("Region == 'Valparaíso'")
indices = pd.to_datetime(covid_valpo["Fecha"], format='%Y-%m-%d')
values = covid_valpo["Casos confirmados"].values
casos = pd.Series(values, index=indices)
casos.head()

Fecha
2020-03-30    7.0
2020-03-30    0.0
2020-03-30    6.0
2020-03-30    0.0
2020-03-30    0.0
dtype: float64

In [4]:
casos_not_null = casos.loc[lambda x: x.notnull()]

In [5]:
casos_min = casos_not_null.values.min()
casos_max = casos_not_null.values.max()
casos_sum = sum(casos_not_null)
casos_mean = casos_sum/(casos_not_null.values.size)

In [6]:
print(f"Mínimo casos: {casos_min}")
print(f"Máximo casos: {casos_max}")
print(f"Suma de casos: {casos_sum}")
print(f"Promedio de casos: {casos_mean}")

Mínimo casos: 0.0
Máximo casos: 7020.0
Suma de casos: 790461.0
Promedio de casos: 373.2110481586402


## Ejercicio 3

* Define el dataframe  `covid_full` uniendo (`merge`) los dataframes `covid_total` y `covid_death` utilizando la _Fecha_ como _key_ y de tal forma de unir solo los registros que se encuentren en ambos conjuntos de datos. Hint: Escoger sabiamente entre _left_, _right_, _inner_ u _outer join_.
* Filtra el dataframe `covid_full` tal que:
    - Fechas con por lo menos 1000 casos totales.
    - Fechas con por lo menos 100 casos nuevos con síntomas.
    - Fechas con más de 20 muertes en el grupo etárea `70-79`.

¿Cuál es la fecha máxima y mínima? ¿Cuántos registros quedan finalmente?

In [7]:
covid_total = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv", usecols=range(3))
covid_total.head()

Fecha  Casos nuevos con sintomas  Casos totales
0  2020-03-02                        1.0            1.0
1  2020-03-03                        0.0            1.0
2  2020-03-04                        2.0            3.0
3  2020-03-05                        1.0            4.0
4  2020-03-06                        1.0            5.0

In [8]:
covid_death = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario_T.csv").rename(columns={"Grupo de edad": "Fecha"})
covid_death.head()

Fecha  <=39  40-49  50-59  60-69  70-79  80-89  >=90
0  2020-04-09     2      1      1      9     21     19     4
1  2020-04-10     2      2      2     10     22     21     6
2  2020-04-11     3      2      2     12     24     24     6
3  2020-04-12     3      2      3     12     28     25     7
4  2020-04-13     3      2      3     12     29     26     7

In [9]:
covid_full = covid_total.merge(covid_death, how="inner", on="Fecha")
covid_full.head()

Fecha  Casos nuevos con sintomas  Casos totales  <=39  40-49  50-59  \
0  2020-04-09                      426.0         5972.0     2      1      1   
1  2020-04-10                      529.0         6501.0     2      2      2   
2  2020-04-11                      426.0         6927.0     3      2      2   
3  2020-04-12                      286.0         7213.0     3      2      3   
4  2020-04-13                      312.0         7525.0     3      2      3   

   60-69  70-79  80-89  >=90  
0      9     21     19     4  
1     10     22     21     6  
2     12     24     24     6  
3     12     28     25     7  
4     12     29     26     7

In [10]:
mask1 = covid_full.loc[lambda x: x["Casos totales"] >= 1000]
mask2 = mask1.loc[lambda x: x["Casos nuevos con sintomas"] >= 100]
mask3 = mask2.loc[lambda x: x["70-79"] > 20]
covid_full_filtered = mask3 
covid_full_filtered

Fecha  Casos nuevos con sintomas  Casos totales  <=39  40-49  50-59  \
0    2020-04-09                      426.0         5972.0     2      1      1   
1    2020-04-10                      529.0         6501.0     2      2      2   
2    2020-04-11                      426.0         6927.0     3      2      2   
3    2020-04-12                      286.0         7213.0     3      2      3   
4    2020-04-13                      312.0         7525.0     3      2      3   
..          ...                        ...            ...   ...    ...    ...   
172  2020-09-28                     1112.0       459671.0   281    430   1283   
173  2020-09-29                     1036.0       461300.0   281    431   1286   
174  2020-09-30                     1185.0       462991.0   281    432   1287   
175  2020-10-01                     1198.0       464750.0   284    435   1296   
176  2020-10-02                     1208.0       466590.0   287    437   1300   

     60-69  70-79  80-89  >=90  
0        9     21     19     4  
1       10     22     21     6  
2       12     24     24     6  
3       12     28     25     7  
4       12     29     26     7  
..     ...    ...    ...   ...  
172   2687   3606   3241  1170  
173   2695   3609   3248  1175  
174   2699   3613   3250  1179  
175   2710   3645   3264  1188  
176   2717   3655   3277  1194  

[177 rows x 10 columns]

In [11]:
covid_full_filtered.index=pd.to_datetime(covid_full_filtered["Fecha"], format='%Y-%m-%d')
print(f"Fecha mínima: {covid_full_filtered.index.min()}")
print(f"Fecha máxima: {covid_full_filtered.index.max()}")
print(f"Cantidad de registros finalmente: {covid_full.index.size}")

Fecha mínima: 2020-04-09 00:00:00
Fecha máxima: 2020-10-02 00:00:00
Cantidad de registros finalmente: 177
